# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [23]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [24]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [25]:
import os
from glob import glob
# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")

In [26]:
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [27]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_px

,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=3377,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,int32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [28]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

C:\Users\chiny\AppData\Local\Temp\ipykernel_17612\3086392435.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [29]:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [30]:
# Add lags for Close and Adj_Close
dd_px['Close_lag_1'] = dd_px['Close'].shift(1)

# Add returns based on Close
dd_px['returns'] = (dd_px['Close'] / dd_px['Close_lag_1']) - 1

# Add the range
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Assign result to dd_feat
dd_feat = dd_px

# Display the first few rows of dd_feat
dd_feat.head()


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range
Ticker,,,,,,,,,,,
A,2001-01-02,30.654194,36.391273,38.537197,35.094776,38.537197,2261684.0,2001,NaN,NaN,3.442421
A,2001-01-03,33.817516,40.146637,40.414879,34.021816,35.139484,4502678.0,2001,36.391273,0.103194,6.393063
A,2001-01-04,35.097927,41.666668,42.694920,39.565453,40.683121,4398388.0,2001,40.146637,0.037862,3.129467
A,2001-01-05,33.177315,39.386623,41.666668,38.268955,40.951359,3277052.0,2001,41.666668,-0.054721,3.397713
A,2001-01-08,32.085224,38.090130,39.878399,37.419529,38.805435,2273288.0,2001,39.386623,-0.032917,2.458870


In [31]:
dd_rets

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
npartitions=3377,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,int32,float64,float64
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [32]:
# Write your code below.

import pandas as pd

pandas_df = dd_feat.compute()

# Calculate 10-day moving average for returns
pandas_df['moving_average_returns'] = pandas_df['returns'].rolling(10).mean()

# Display the first few rows of the DataFrame
print(pandas_df.head())


Price        Date  Adj Close      Close       High        Low       Open  \
Ticker                                                                     
A      2001-01-02  30.654194  36.391273  38.537197  35.094776  38.537197   
A      2001-01-03  33.817516  40.146637  40.414879  34.021816  35.139484   
A      2001-01-04  35.097927  41.666668  42.694920  39.565453  40.683121   
A      2001-01-05  33.177315  39.386623  41.666668  38.268955  40.951359   
A      2001-01-08  32.085224  38.090130  39.878399  37.419529  38.805435   

Price      Volume  Year  Close_lag_1   returns  hi_lo_range  \
Ticker                                                        
A       2261684.0  2001          NaN       NaN     3.442421   
A       4502678.0  2001    36.391273  0.103194     6.393063   
A       4398388.0  2001    40.146637  0.037862     3.129467   
A       3277052.0  2001    41.666668 -0.054721     3.397713   
A       2273288.0  2001    39.386623 -0.032917     2.458870   

Price   moving_average_re

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No, Dask can do it as well.

+ Would it have been better to do it in Dask? Why?
Yes, it saves computational time and resources for large data sets such as these.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.